# Setup COCO API


In [1]:
!pip install Cython
!pip install -q "tqdm>=4.36.1"
!pip3 install xgboost
!pip3 install scikit-image scikit-learn graphviz

In [2]:
!git clone https://github.com/waleedka/coco
!pip install -U setuptools
!pip install -U wheel
!make install -C coco/PythonAPI

fatal: destination path 'coco' already exists and is not an empty directory.
Requirement already up-to-date: setuptools in /opt/conda/lib/python3.7/site-packages (50.3.2)
Requirement already up-to-date: wheel in /opt/conda/lib/python3.7/site-packages (0.35.1)
make: Entering directory '/home/jupyter/modified-FCOS/DTL/coco/PythonAPI'
# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
building 'pycocotools._mask' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
creating build/common
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.7/site-packages/numpy/core/include -I../common -I/opt/conda/include/python3.7m -c pycocotools/_mask.c -o build/temp.linux-x86_64-3.7/pycocotools/_mask.o -Wno-cpp -Wno-unused-function -std=c99
^C
interrupted
Makefile:7: recipe for target 'install' 

In [3]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip ./annotations_trainval2017.zip

--2020-11-06 01:44:33--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.88.163
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.88.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

      annotations_t   0%[                    ] 223.26K   263KB/s               ^C
Archive:  ./annotations_trainval2017.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./annotations_trainval2017.zip or
        ./annotations_trainval2017.zip.zip, and cannot find ./annotations_trainval2017.zip.ZIP, period.


# Import needed and Load COCO categories

In [5]:
from pycocotools.coco import COCO
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.util import crop
from skimage import io, color
from math import floor, ceil
from PIL import Image
from joblib import dump, load
from tqdm.notebook import tqdm
from os import path
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz 
import json
import pandas as pd

In [6]:
def split_integer(num):
    return (floor(num/2), ceil(num/2))

def create_dataset_id(coco):
    catIds = coco.getCatIds(catNms=['dog','cat'])
    img_catdog = coco.getImgIds(catIds=catIds)
    img_dog = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=['dog']))) - set(img_catdog))
    img_cat = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=['cat']))) - set(img_catdog))
    print("Dog Image Total: ",len(img_dog),"\nCat Image Total: ", len(img_cat))
    return img_dog + img_cat, np.array(len(img_dog) * [1] + len(img_cat) * [0])

def load_resize_crop_img(img_url, target_width=64, rgb = True):
    I = io.imread(img_url)
    factor = target_width/min(I.shape[0:2])
    resize_target = (round(I.shape[0] * factor), round(I.shape[1] * factor))
    resized_image = (resize(I, resize_target, anti_aliasing=False) * 255).astype(np.uint8)
    crop_width = (split_integer(resize_target[0] - target_width), split_integer(resize_target[1] - target_width), (0,0))
    if rgb:
        cropped_image = crop(color.gray2rgb(resized_image), crop_width)
    else:
        cropped_image = crop(color.rgb2gray(resized_image), crop_width)
    return cropped_image

def flatten_img(img):
    return img.flatten()

In [57]:
train_path="annotations/instances_train2017.json"
val_path="annotations/instances_val2017.json"
annotation = {"train":train_path, "val":val_path}

N_TRAIN = 4000
IMG_SIZE = 64



*   Gabungin Data training dan validasi,
*   Cari tau kalau di DTL, Random Forest sama XGBoost ada train/val/test ga?
- Apakah urutan pengaruh untuk hasil?
- Bikin fungsi buat evaluate hasilnya ke set tertentu. Coba cari tau evaluation metrics apa yang relevan buat ini  di SKLearn



# Get COCO Image Data

In [8]:
def get_coco_id(annotation_file, cat1 = "dog", cat2 = "cat", n = None):
    coco = COCO(annotation_file)
    catIds = coco.getCatIds(catNms=[cat1,cat2])
    img_combine = coco.getImgIds(catIds=catIds)
    img_cat1 = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=[cat1]))) - set(img_combine))
    img_cat2 = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=[cat2]))) - set(img_combine))
    if n is None:
        return img_cat1 + img_cat2, np.array(len(img_cat1) * [1] + len(img_cat2) * [0])
    return img_cat1[:n] + img_cat2[:n], np.array(len(img_cat1[:n]) * [1] + len(img_cat2[:n]) * [0])

In [54]:
X_train, y_train = get_coco_id(annotation["train"], n=N_TRAIN)
X_val, y_val = get_coco_id(annotation["val"])

loading annotations into memory...
Done (t=15.57s)
creating index...
index created!
loading annotations into memory...
Done (t=0.40s)
creating index...
index created!


In [16]:
coco = COCO(annotation["train"])
for x in tqdm(X_train):
    fn = "train/{}.png".format(x)
    if path.exists(fn):
#         print(f"skipping {x}")
        continue
    I = io.imread(coco.loadImgs(x)[0]['coco_url'])
    io.imsave(fn,I)

loading annotations into memory...
Done (t=18.34s)
creating index...
index created!


In [17]:
coco = COCO(annotation["val"])
for x in tqdm(X_val):
    fn = "val/{}.png".format(x)
    if path.exists(fn):
#         print(f"skipping {x}")
        continue
    I = io.imread(coco.loadImgs(x)[0]['coco_url'])
    io.imsave(fn,I)

loading annotations into memory...
Done (t=1.89s)
creating index...
index created!


## Preprocess the image and flatten it

In [55]:
# coco = COCO(annotation["train"])
X_loaded = []
for x in tqdm(X_train):
    try : 
        fn = "train/{}.png".format(x)
        X_loaded += [load_resize_crop_img(fn, target_width=IMG_SIZE).flatten()]
    except :
        print(x)
print("Finished with length: ",len(X_loaded))
X_loaded = np.array(X_loaded)
# df_train = pd.Dataframe({"X":X_loaded,"y":y_train})
print("Converted to ndarray with shape: ",X_loaded.shape, y_train.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass-through of possibly RGB images in gray2rgb is deprecated. In version 0.19, input arrays will always be considered grayscale, even if the last dimension has length 3 or 4. To prevent this warning and ensure compatibility with future versions, detect RGB images outside of this function.



Finished with length:  8083
Converted to ndarray with shape:  (8083, 12288) (8083,)


In [56]:
# coco = COCO(annotation["val"])
X_loaded_val = []
for x in tqdm(X_val):
    try : 
        fn = "val/{}.png".format(x)
        X_loaded_val += [load_resize_crop_img(fn, target_width=IMG_SIZE).flatten()]
    except :
        print(x)
print("Finished with length: ",len(X_loaded_val))
X_loaded_val = np.array(X_loaded_val)
print("Converted to ndarray with shape: ",X_loaded_val.shape, y_val.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass-through of possibly RGB images in gray2rgb is deprecated. In version 0.19, input arrays will always be considered grayscale, even if the last dimension has length 3 or 4. To prevent this warning and ensure compatibility with future versions, detect RGB images outside of this function.



Finished with length:  337
Converted to ndarray with shape:  (337, 12288) (337,)


## Prepare the fold for train - validate GridSearch

In [58]:
test_fold = np.concatenate([
    # The training data.
    np.full(len(X_train), -1, dtype=np.int8),
    # The development data.
    np.zeros(len(X_val), dtype=np.int8)
])
cv = PredefinedSplit(test_fold)

X = np.concatenate([X_loaded, X_loaded_val])
y = np.concatenate([y_train, y_val])

# dtrain = xgb.DMatrix(X_loaded, label=y_train)
# dval = xgb.DMatrix(X_loaded_val, label=y_val)

### Save features and label for use later

In [60]:
np.save("coco_dog_cat_feature_{}_{}.npy".format(N_TRAIN, IMG_SIZE), X)
np.save("coco_dog_cat_label_{}_{}.npy".format(N_TRAIN, IMG_SIZE), y)

In [61]:
# Magic to delete unneeded variable (in this case the features array and label array) as it has been aggregated to X and y respectively
# %reset_selective "^X_loaded(_train)?$"
# %reset_selective "^y_(train|val)$"

Once deleted, variables cannot be recovered. Proceed (y/[n])?   y
Once deleted, variables cannot be recovered. Proceed (y/[n])?   y


# Grid Search "Training"
## Decision Tree Learning 

In [62]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100)

In [63]:
model = model.fit(X,y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed:  2.6min remaining:  5.3min
[Parallel(n_jobs=4)]: Done   3 out of   6 | elapsed:  2.6min remaining:  2.6min
[Parallel(n_jobs=4)]: Done   4 out of   6 | elapsed:  2.7min remaining:  1.3min
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  5.1min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  5.1min finished


In [64]:
print(model.cv_results_)

{'mean_fit_time': array([105.8601377 , 154.87533522, 155.65522337, 155.00233984,
       165.83737779, 150.17876673]), 'std_fit_time': array([0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.01488638, 0.01361084, 0.01064777, 0.01514888, 0.01022601,
       0.00982618]), 'std_score_time': array([0., 0., 0., 0., 0., 0.]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[10, 100, 1000, 10, 100, 1000],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'criterion': 'gini', 'max_depth': 10}, {'criterion': 'gini', 'max_depth': 100}, {'criterion': 'gini', 'max_depth': 1000}, {'criterion': 'entropy', 'max_depth': 10}, {'criterion': 'entropy', 'max_depth': 100}, {'criterion': 'entropy', 'max_depth': 1000}], 

In [65]:
dump(model, 'gs_dtl_{}_{}.pkl'.format(N_TRAIN, IMG_SIZE))

['gs_dtl_4000_64.pkl']

In [ ]:
%reset_selective "^model$"

## Random Forest Classifier

In [66]:
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100)

In [67]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   24.2s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   44.5s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   47.4s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  2.3min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  4.5min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  4.8min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:  5.4min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:  6.0min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  6.5min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  6.9min
[Parallel(

In [73]:
print(rf_model.cv_results_)
dump(rf_model, 'gs_rf_{}_{}.pkl'.format(N_TRAIN, IMG_SIZE))

{'mean_fit_time': array([ 21.78393483,  42.09192157,  66.15766811, 265.10303378,
        23.05242348,  59.69388962, 104.70965838, 467.23194075,
        32.68051052, 109.91214442, 205.3528204 , 874.84899735,
        19.28056598,  38.74321795,  63.64638782, 270.90702176,
        23.06845093,  59.23746562, 106.32675576, 453.62110424,
        33.36874604, 109.94556618, 206.16823339, 730.08133388]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.15431142, 0.14929819, 0.15170312, 0.15451217, 0.13797402,
       0.1436832 , 0.12515092, 0.14837337, 0.12343049, 0.12434363,
       0.12623191, 0.08292913, 0.14132214, 0.12437248, 0.12249684,
       0.12707281, 0.12272716, 0.12494969, 0.14289641, 0.09648132,
       0.138412  , 0.12769341, 0.12619448, 0.08285069]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 

['gs_rf_4000_64.pkl']

In [ ]:
%reset_selective "^rf_model$"

In [70]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [4, 6, 10],
"learning_rate" : [0.1, 0.3],
"n_estimators" : [1, 5, 10, 50]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100)

In [71]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   24.8s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   50.3s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   50.3s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:  5.4min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  5.7min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:  6.5min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:  7.5min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  7.8min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  8.2min
[Parallel(

In [72]:
print(xg_model.cv_results_)
dump(xg_model, 'gs_xg_{}_{}.pkl'.format(N_TRAIN, IMG_SIZE))

{'mean_fit_time': array([  22.43522739,   47.95351934,   79.80628848,  319.83033085,
         25.29649806,   72.8796227 ,  130.89652324,  556.08952594,
         40.80411887,  140.88618803,  266.77442312, 1085.70730758,
         20.28133869,   46.10543799,   77.24842286,  313.83477449,
         25.53003454,   70.00771546,  129.9080739 ,  523.08644485,
         40.23054242,  141.34746671,  263.67096138,  808.37351775]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.15814829, 0.15463328, 0.15414119, 0.15768194, 0.16423941,
       0.12761998, 0.14999938, 0.16115379, 0.12289   , 0.12598634,
       0.12820029, 0.08213401, 0.14898276, 0.12322354, 0.12716866,
       0.13542223, 0.12214041, 0.12492013, 0.12589145, 0.11032391,
       0.12317371, 0.12402463, 0.12928724, 0.08245492]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.

['gs_xg_4000_64.pkl']